### Import library
Nous allons importer les bibliothèque numpy et pandas qui sont necessaire pour le traitement.

In [2]:
import pandas as pd
import numpy as np

### Import Data
Nous allons importer les données depuis le répertoire RAwData pour enfin les traiter et transformer

In [130]:
n=['TimesTamp', 'ECP', 'Antenna', 'RSSI', 'FP']
#p='../2-TransformedData/dataFinale/mergedFiles/all-3/p25.csv'
#p='../2-TransformedData/dataFinale/mergedFiles/all-3/p27.csv'
p='../2-TransformedData/dataFinale/mergedFiles/all-3/p30.csv'
d=','
def importData(path, delimit,cols):
    return pd.read_csv(path, sep=delimit,names=cols)

data=importData(p,d,n)

print(data.shape)
data.describe()

(19591, 5)


,TimesTamp,Antenna,RSSI,FP
count,1.959100e+04,19591.000000,19591.000000,19591.000000
mean,1.581422e+15,2.136185,-61.628886,0.308713
std,5.543056e+09,0.947328,4.260234,0.461974
min,1.581414e+15,1.000000,-79.000000,0.000000
25%,1.581415e+15,1.000000,-64.000000,0.000000
50%,1.581421e+15,2.000000,-61.500000,0.000000
75%,1.581428e+15,3.000000,-59.000000,1.000000
max,1.581430e+15,4.000000,-42.000000,1.000000


### TYPAGE DES CHAMPS
Nos allons typer les différentes colonnes du dataSet avec les types primaires de python 

In [131]:
def typage(data):
    data['ECP']=data['ECP'].astype(str)
    data['TimesTamp']=data['TimesTamp'].astype('int64')
    data['RSSI']=data['RSSI'].astype('float64')
    data['Antenna']=data['Antenna'].astype('int64')
    data['FP']=data['FP'].astype('int64')
    return data

data=typage(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19591 entries, 0 to 19590
Data columns (total 5 columns):
TimesTamp    19591 non-null int64
ECP          19591 non-null object
Antenna      19591 non-null int64
RSSI         19591 non-null float64
FP           19591 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 765.4+ KB


### Build DataSet
Nous allons contruire la dataFrame avec les colonnes

In [132]:
from scipy.stats import variation 

def generateDataSet(data, groupedBy):
    grouped_df = data.groupby(groupedBy) #On groupe les données par ..."Etiquetes"
    
    dataSet = pd.DataFrame({}) #Création d'une nouvelle dataFrame
    
    #Création des variables intermédiaire pour la récupération des champs
    ecp=pd.Series(grouped_df.ECP.unique().index,name="ECP")
    fp=pd.Series(grouped_df.FP.max(),name="FP")
    rc= pd.Series(grouped_df.ECP.count(),name="readcount")
    minRSSI= pd.Series(grouped_df.RSSI.min(),name="minRssi")
    maxRSSI= pd.Series(grouped_df.RSSI.max(),name="maxRssi")
    mean=pd.Series(grouped_df.RSSI.mean(),name="meanRssi")
    startTime=pd.Series(grouped_df.TimesTamp.min(),name="startTime")
    endTime=pd.Series(grouped_df.TimesTamp.max(),name="endTime")
    diff=pd.Series(grouped_df.RSSI.diff,name="difference")
    
    #Mise à jour des champs de la dataFrame
    dataSet['ECP']=ecp.values
    dataSet['RC']=rc.values
    dataSet['MIN']=minRSSI.values
    dataSet['MAX']=maxRSSI.values
    dataSet['MEAN']=mean.values
    dataSet['START']=startTime.values
    dataSet['END']=endTime.values
    dataSet['DURATION']=endTime.values-startTime.values
    cv={}
    a1={}
    a2={}
    a3={}
    a4={}
    for key, item in grouped_df:
        cv[key]=variation(grouped_df['RSSI'].get_group(key).values,axis=0)
        a1[key]=0
        a2[key]=0
        a3[key]=0
        a4[key]=0
        for a in grouped_df['Antenna'].get_group(key).values:
            if(a==1): a1[key]+=1
            if(a==2): a2[key]+=1
            if(a==3): a3[key]+=1
            if(a==4): a4[key]+=1
                
    dataSet['CV']=pd.Series(cv).values
    dataSet['A1']=pd.Series(a1).values
    dataSet['A2']=pd.Series(a2).values
    dataSet['A3']=pd.Series(a3).values
    dataSet['A4']=pd.Series(a4).values
    dataSet['FP']=fp.values
    
    return dataSet

dataSet=generateDataSet(data,'ECP')
print(len(dataSet))
dataSet.tail(10)

2634


,ECP,RC,MIN,MAX,MEAN,START,END,DURATION,CV,A1,A2,A3,A4,FP
2624,E28068900000000018345A9A20,4,-75.0,-74.0,-74.750000,1581413594441149,1581413600049315,5608166,-0.005793,0,0,0,4,1
2625,E28068900000000018345BD020,15,-70.0,-67.0,-69.233333,1581413593450272,1581413600573550,7123278,-0.009459,0,0,0,15,1
2626,E28068900000000018345CCE89,4,-74.0,-73.5,-73.875000,1581427933174550,1581427940458432,7283882,-0.002931,0,0,4,0,1
2627,E28068900000000018345CCE90,9,-75.0,-74.0,-74.277778,1581427892763170,1581427894629483,1866313,-0.004611,0,0,9,0,1
2628,E28068900000000018345CCE91,1,-73.5,-73.5,-73.500000,1581427918401483,1581427918401483,0,-0.000000,0,0,1,0,1
2629,E28068900000000018345CCE96,7,-74.5,-74.0,-74.428571,1581426501924322,1581426507565758,5641436,-0.002351,0,0,7,0,1
2630,E28068900000000018345CD520,82,-64.0,-61.0,-62.750000,1581413593128082,1581413603380191,10252109,-0.010795,0,42,0,40,1
2631,E28068900000000018345CD597,7,-71.5,-68.0,-70.214286,1581429783597354,1581429788167062,4569708,-0.017384,3,4,0,0,1
2632,E28068900000000018345CD598,25,-72.0,-63.0,-68.360000,1581429617114063,1581429622250188,5136125,-0.031205,10,10,5,0,1
2633,E28068900000000018345DFC20,9,-71.0,-70.0,-70.611111,1581413593309779,1581413598895958,5586179,-0.005563,0,0,9,0,1


### Export Data
Nous allons sauvegarder les donées dans un fichier .csv

In [133]:
import time
def exportData(data, path):
    data.to_csv(path, index = None, header=False)

## Export allinOne
Exporter l'ensemble des daonnées dans un seul.

In [129]:
tm = time.strftime("%Y-%b-%d-%H_%M_%S")
#path='../3-DataSet/dataFinale/p25/dataSet_p25_'+tm+'.csv'
#path='../3-DataSet/dataFinale/p25/dataSet_p25.csv'
#path='../3-DataSet/dataFinale/p27/dataSet_p27.csv'
path='../3-DataSet/dataFinale/p30/dataSet_p30.csv'
exportData(dataSet, path)

### Split Data
Nous allons découper les données en deux, 80% des trainSets et 20% des tesSets

In [141]:
def splitDataLE20(data):
    dataSet_np = np.random.rand(len(data)) <0.8
    trainSet=data[dataSet_np]
    testSet=data[~dataSet_np]
    return trainSet, testSet

from sklearn.model_selection import train_test_split
def sliptDataE20(data):
    X = data.iloc[:, :-1].values
    y = data['FP'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    trainSet=pd.DataFrame(X_train)
    trainSet[13]=y_train
    
    testSet = pd.DataFrame(X_test)
    testSet[13] = y_test
    return trainSet, testSet

### Export Train & Test
Nous allons exporter les données découpées en traint et test

In [142]:
pathTrain='../3-DataSet/dataFinale/p30/traintSet_p30.csv'
pathTest='../3-DataSet/dataFinale/p30/testSet_p30.csv'

trainSet, testSet=sliptDataE20(dataSet)
#trainSet, testSet=splitDataLE20(dataSet)

exportData(trainSet, pathTrain)
exportData(testSet, pathTest)